# **REMOTE PHOTOPLETHYSMOGRAPHY (rPPG)**

## IMPORTS Y CREACIÓN DEL ENTORNO PARA EL PROYECTO

Se va a crear un nuevo environment utilizando **_Anaconda Prompt_** con las dependencias necesarias y únicas para la correcta realización del proyecto.

Ya con las instalaciones y el environment creado, se va a proceder a la importación de las librerías necesarias para el proyecto.

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time
from scipy import signal
from scipy.fft import rfft, rfftfreq

## REMOTE HEARD FRECUENCY

Se va a capturar la frecuencia cardiaca a través de la WebCam del dispositivo.

**Limitaciones y Consideraciones técnicas:**

Aunque el sistema implementa algoritmos robustos como **POS (Plane-Orthogonal-to-Skin)** y segmentación **Multi-ROI**, la fotoplestimografía remota presenta limitaciones inherentes que dependen del entorno y la fisiología del sujeto.

- **Iluminación y Relación Señal-Ruido (SNR):** El sistema no mide directamente el flujo sanguíneo, sino las variaciones sutiles en la absorción de luz. Por tanto, la calidad de la señal depende estrictamente de la fuente lumínica:

    - **Baja iluminación:** El sensor de la cámara aumenta ganancia (ISO) automáticamente, introduciendo ruido de disparo (shot noise) y ruido térmico granular que enmascara la componente AC (pulsátil) de la señal cardíaca.

    - **_Aliasing_ por iluminación artificial:** Fuentes de luz fluorescente aintiguas con parpadeo perceptible (50/60Hz) pueden introducir armónicos en el espectro de frecuencias que, por *aliasing* se podrían interpretar como latidos cardíacos si no se filtran adecuadamente.

- **Variabilidad fisiológica y fototipos de piel:** La eficacia del algoritmo varía según diferentes grupos étnicos y características físicas del sujeto:

    - **Fototipos altos:** Las pieles con mayor concentración de melanina absorben mayor cantidad de luz, reduciendo así la intensidad de la luz reflejada y disminuyendo en consecuencia la amplitud de la señal PPG. El resultado de lo anterior es que la **Relación Señal-Ruido (SNR)** es más baja comparada a la obtenida en pieles claras. Como solución, se puede implementar mayor iluminación o que la misma sea más intensa para obtener resultados fiables.

    - **Oclusiones faciales:** El modelo utiliza tanto la frente como las mejillas, por lo tanto, si el individuo posee vello facial denso, gafas de montura muy gruesa, flequillos o cualquier objeto que interfiera directamente en estas zonas, se reduce el área efectiva de la ROI (Región de Interés).

- **Artefactos de movimiento:** Aunque se implementa el algormito de proyección ortogonal (POS), si se suceden movimientos bruscos como hablar o gesticular se introducen cambios no lineales en la reflexión de la luz que no pueden ser cancelados totalmente por el modelo matemático lineal.

    - **Restricción:** El individuo debe mantenerse a una posición relativamente estática para garantizar una precisión óptima.

- **Latencia en la inicialización:** El sistema hace uso de un buffer temporal para realizar el análisis en el dominio de la frecuencia (FFT).

    - **Cold Start:** El sistema tiene un retardo inicial de aproximadamente 5/6 segundos (viene dado por el tamaño del buffer) desde que se detecta la cara hasta que se encuentra el primer dato fiable, impidiendo así la medición instantánea.

- **Limitaciones Hardware:**

    - **Compresión de Video:** Muchas webcams comprimen el video en tiempo real. Los artefactos de compresión pueden alterar lso valores de color de los píxeles, suavisando los micro-cambios de color necesarios para la rPPG.

    - **Jitter de FPS:** Si la CPU está saturada, la tasa de frames puede no ser constante. Aunque el código recalcula lso FPS dinámicamente, caídas drásticas afectarán la precisión de la **Transformada de Fourier**.

In [10]:
class RobustHeartRateDetector:
    def __init__(self):
        # --- CONFIGURACIÓN DE MEDIAPIPE ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=False,
            min_detection_confidence=0.6,
            min_tracking_confidence=0.6
        )
        
        # --- PARÁMETROS DEL SISTEMA ---
        self.BUFFER_SIZE = 160     # Ventana de ~5.3 segundos a 30fps
        self.FS = 30.0             # Frecuencia de muestreo estimada (se recalcula)
        
        # Buffers de datos
        self.rgb_buffer = []       # Almacena tuplas (R, G, B) promedio
        self.times_buffer = []     # Tiempos de captura
        self.bpm_history = []      # Para suavizar la lectura final
        
        self.current_bpm = 0.0
        self.is_stable = False     # Bandera para indicar si la lectura es fiable

    def setup_camera(self):
        """Intenta configurar la cámara para desactivar auto-ajustes."""
        cap = cv2.VideoCapture(0)
        
        # Intentar desactivar auto-exposición (0.25/0.75 suele ser manual en algunas cams)
        # Nota: Esto varía mucho según la marca de la webcam.
        cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25) 
        cap.set(cv2.CAP_PROP_EXPOSURE, -5.0) # Valor bajo para evitar saturación
        
        # Desactivar auto-foco
        cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
        
        return cap

    def get_multi_roi_color(self, frame, landmarks):
        """
        Extrae el promedio RGB combinado de Frente y Mejillas.
        Usa una máscara para ser más preciso.
        """
        h, w, c = frame.shape
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Definir polígonos de las ROIs usando índices de MediaPipe
        # Frente, Mejilla Izq, Mejilla Der
        rois_indices = [
            [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109], # Contorno cara simplificado
            [151, 108, 69, 104, 68, 71],    # Frente (aprox)
            [330, 347, 346, 352],           # Mejilla Izq
            [101, 118, 117, 123]            # Mejilla Der
        ]
        
        # Dibujar ROIs visuales (solo mejillas y frente para simplificar la media)
        # Índices específicos para obtener piel limpia
        roi_points_list = [
            [109, 69, 104, 108, 151, 337, 299, 333, 298], # Frente extendida
            [266, 425, 435, 346, 347, 329, 371],          # Mejilla Izquierda
            [36, 205, 215, 117, 118, 100, 142]            # Mejilla Derecha
        ]
        
        combined_mask = np.zeros((h, w), dtype=np.uint8)
        
        for region in roi_points_list:
            points = np.array([[int(landmarks.landmark[i].x * w), int(landmarks.landmark[i].y * h)] for i in region], np.int32)
            cv2.fillPoly(combined_mask, [points], 255)
            # Dibujar contorno verde en el frame para feedback
            cv2.polylines(frame, [points], True, (0, 255, 0), 1)

        # Extraer promedio de color donde la máscara es blanca
        mean_color = cv2.mean(frame, mask=combined_mask)[:3] # (B, G, R)
        return mean_color # Devuelve (B, G, R)

    def pos_algorithm(self, rgb_array):
        """
        Implementación robusta del algoritmo POS (Wang et al., 2016).
        Proyecta la señal RGB en un plano ortogonal al tono de piel para eliminar movimiento.
        """
        # Entrada: rgb_array shape (N_frames, 3) -> Convertir a (3, N_frames)
        C = rgb_array.T 
        
        # 1. Normalización temporal (Dividir por la media para quitar brillo DC)
        mean_color = np.mean(C, axis=1, keepdims=True)
        Cn = C / (mean_color + 1e-6) - 1
        
        # 2. Proyección (Matemática de POS)
        # S1 = G - B
        # S2 = G + B - 2R
        # Ojo: OpenCV entrega BGR, así que índices: 0=B, 1=G, 2=R
        S1 = Cn[1, :] - Cn[0, :]
        S2 = Cn[1, :] + Cn[0, :] - 2 * Cn[2, :]
        
        # 3. Fusión Alpha
        std_s1 = np.std(S1)
        std_s2 = np.std(S2)
        alpha = std_s1 / (std_s2 + 1e-6)
        
        P = S1 + alpha * S2
        
        return P

    def compute_heart_rate(self):
        if len(self.rgb_buffer) < self.BUFFER_SIZE:
            return

        # Convertir buffer a array
        rgb_data = np.array(self.rgb_buffer) # (Frames, 3)
        times = np.array(self.times_buffer)
        
        # Calcular FPS reales dinámicos
        elapsed = times[-1] - times[0]
        fs = len(times) / elapsed if elapsed > 0 else 30.0
        
        # --- A. Aplicar Algoritmo POS ---
        ppg_signal = self.pos_algorithm(rgb_data)
        
        # --- B. Filtrado (Bandpass Butterworth) ---
        # Filtramos entre 0.7 Hz (42 BPM) y 3.5 Hz (210 BPM)
        nyquist = fs / 2
        low = 0.7 / nyquist
        high = 3.5 / nyquist
        b, a = signal.butter(3, [low, high], btype='band')
        filtered_ppg = signal.filtfilt(b, a, ppg_signal)
        
        # --- C. FFT (Fourier) ---
        N = len(filtered_ppg)
        yf = rfft(filtered_ppg)
        xf = rfftfreq(N, 1 / fs)
        
        # Máscara para rango humano
        mask = (xf >= 0.7) & (xf <= 3.5)
        valid_xf = xf[mask]
        valid_yf = np.abs(yf[mask])
        
        if len(valid_yf) > 0:
            peak_idx = np.argmax(valid_yf)
            freq_hz = valid_xf[peak_idx]
            new_bpm = freq_hz * 60
            
            # Suavizado de la lectura (Media móvil)
            self.bpm_history.append(new_bpm)
            if len(self.bpm_history) > 10: # Guardar últimos 10 cálculos
                self.bpm_history.pop(0)
            
            self.current_bpm = np.mean(self.bpm_history)
            self.is_stable = True

    def run(self):
        cap = self.setup_camera()
        print("Iniciando Sistema Biométrico Robusto...")
        print("Manten tu cara iluminada y dentro de los recuadros verdes.")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            curr_time = time.time()
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame)

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # 1. Obtener color promedio (Frente + Mejillas)
                    # Ojo: get_multi_roi_color devuelve BGR porque usa 'frame' original
                    bgr_val = self.get_multi_roi_color(frame, face_landmarks)
                    
                    # 2. Actualizar Buffers
                    self.rgb_buffer.append(bgr_val)
                    self.times_buffer.append(curr_time)
                    
                    # Mantener tamaño fijo (FIFO)
                    if len(self.rgb_buffer) > self.BUFFER_SIZE:
                        self.rgb_buffer.pop(0)
                        self.times_buffer.pop(0)
                    
                    # 3. Procesar señal (cada X frames para ahorrar CPU)
                    if len(self.rgb_buffer) == self.BUFFER_SIZE and len(self.rgb_buffer) % 2 == 0:
                        self.compute_heart_rate()

                    # --- INTERFAZ VISUAL ---
                    # Barra de progreso de captura
                    if not self.is_stable:
                        progress = int((len(self.rgb_buffer) / self.BUFFER_SIZE) * 100)
                        cv2.putText(frame, f"Calibrando: {progress}%", (20, 50), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                    else:
                        # Mostrar BPM
                        color_bpm = (0, 255, 0) if 60 < self.current_bpm < 100 else (0, 165, 255)
                        cv2.putText(frame, f"BPM: {self.current_bpm:.1f}", (20, 60), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, color_bpm, 3)
                        cv2.putText(frame, "Algoritmo: POS + Multi-ROI", (20, 90), 
                                    cv2.FONT_HERSHEY_PLAIN, 1, (200, 200, 200), 1)

            else:
                # Resetear si se pierde la cara
                self.rgb_buffer = []
                self.times_buffer = []
                self.is_stable = False
                cv2.putText(frame, "NO FACE DETECTED", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.imshow('TFG - Biometria Avanzada', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    app = RobustHeartRateDetector()
    app.run()

Iniciando Sistema Biométrico Robusto...
Manten tu cara iluminada y dentro de los recuadros verdes.
